# 情報データ科学演習III第3週：画像のフィルタリング


第3週では，Pythonで読み込んだ画像に対してフィルタリングを適用する処理方法を習得する．




まずいろいろなモジュールをインポートするために以下のセルを実行する．notebookを起動したときには，これらのモジュールをインポートしなければ，これらのモジュールの関数を使うことはできない．

In [ ]:
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline
plt.gray();
from matplotlib.pyplot import imshow

from skimage.io import imread, imsave
from skimage.color import rgb2gray

from scipy.signal import fftconvolve as convolve

# ===================================


# パート1：空間フィルタリング

ここでは，画像のボケに代表される平均値フィルタによるフィルタリングの例を見る．


## 平均値フィルタ

平滑化フィルタは画像をぼかしたりノイズを減らすために用いられる．平滑化フィルタの出力は，フィルタマスクの近傍にある画素値の平均である．
こうすることによって，出力画像では輝度の急激な変化が少なくなり，画像をぼかすことができる．

次式がサイズが3×3の平滑化フィルタである．
フィルタの中央が対応する画素に相当し，その周囲8画素（とその画素自身）の輝度値を，
フィルタの数値を重みにして平均を計算する．

$$
W =
\frac{1}{9}
\begin{pmatrix}
1 & 1 & 1\\
1 & 1 & 1\\
1 & 1 & 1
\end{pmatrix}
$$

`N`が奇数の場合にも，同様にNxNの平均値フィルタが設定できる．`N`を大きくするほど，平均を計算する近傍が大きくなる．

In [ ]:
N = 3
W = np.ones((N, N)) / (N**2)  # N×N平滑化フィルタW
print(W)

平均値フィルタを画像に適用するには`convolve`を用いる．

引数：

- `im`：フィルタリングする対象の画像
- `W`：フィルタ
- `mode`：画像の端の部分の処理の種類．通常は`mode='same'`とする．

返り値：

- フィルタリングされた画像


使用例：
- `convolve(im, W, mode='same')`

以下の2枚の画像をフィルタリングする．

<img src="images/test_pattern.jpg" width=400>
<center>図：test_pattern.jpg
(<a href="https://commons.wikimedia.org/wiki/File:EIA_Resolution_Chart_1956.svg#/media/File:EIA_Resolution_Chart_1956.svg">By Self painted - Self painted, Public Domain, <a href="https://commons.wikimedia.org/w/index.php?curid=6824470">Link</a>)
</center>

<img src="images/hubble.jpg" width=400>
<center>図：hubble.jpg
(public domain from wikimedia, 
<a href="https://ja.wikipedia.org/wiki/ファイル:NASA-HS201427a-HubbleUltraDeepField2014-20140603.jpg">"Hubble Team Unveils Most Colorful View of Universe Captured by Space Telescope"</a>)
</center>

## グレースケール画像の場合

グレースケール画像に対するフィルタリングは，convolveを適用するだけである．

以下では，対象となる画像が`im`，適用するフィルタが`W`である．


In [ ]:
im = rgb2gray(imread(os.path.join('images', 'test_pattern.jpg')))  # グレースケール画像

im *= 255  # rgb2grayは画素値の範囲を0から1に変換してしまうため，255をかけておく

plt.figure(figsize=(20, 20))
imshow(im)
plt.title('original image')
plt.show()

N = 15
W = np.ones((N, N)) / (N**2)  # N×N平滑化フィルタW
im_filtered = convolve(im, W, mode='same')

plt.figure(figsize=(20, 20))
imshow(im_filtered)
plt.title('filtered image')
plt.show()

imsave('IP3-filtering.jpg', im_filtered.astype(np.uint8))  # 処理画像の保存

## カラー画像の場合

カラー画像の場合には，RGBの各チャンネルに対して適用する．

カラー画像`im`のi番目のチャンネルを取り出すには，`im[:, :, i]`と指定する．これはグレースケール画像であり，これに対してフィルタリングすれば良いので，`convolve(im[:, :, i], W, mode='same')`とする．

その結果はi番目のチャンネルの結果なので，あらかじめ準備しておいた受け取り用配列のi番目のチャンネル
`im_filtered[:, :, i]`に代入する．

In [ ]:
im = imread(os.path.join('images', 'hubble.jpg'))  # カラー画像

plt.figure(figsize=(20, 20))
imshow(im)
plt.title('original image')
plt.show()

N = 15
W = np.ones((N, N)) / (N**2)  # N×N平滑化フィルタW
im_filtered = np.zeros_like(im)  # imと同じサイズの配列を準備（値はすべて0）

for i in range(3):  # RGBそれぞれに適用
    # i番目のチャンネルについて
    im_filtered[:, :, i] = convolve(im[:, :, i], W, mode='same')

plt.figure(figsize=(20, 20))
imshow(im_filtered)
plt.title('filtered image')
plt.show()

imsave('IP3-filtering.jpg', im_filtered.astype(np.uint8))  # 処理画像の保存


## レポート課題1

- 画像`test_pattern.jpg`, `hubble.jpg`を読み込み，それぞれに対して平均値フィルタによるフィルタリングを適用する．
- 平均値フィルタのサイズ`N`（奇数）を3, 5, ..., 55まで変えて，結果画像のどの部分がどのようにぼかされているのかを考察する．



## レポート課題2

平均値フィルタの代わりに，以下のフィルタを適用する．



In [ ]:
N = 3
W1 = np.eye(N) / N
print(W1)

- 画像`test_pattern.jpg`, `hubble.jpg`を読み込み，それぞれに対してこのフィルタによるフィルタリングを適用する．
- フィルタのサイズ`N`（奇数）を3, 5, ..., 55まで変えて，結果画像のどの部分がどのようになっているのか，なぜそうなるのか，また平均値フィルタとどのように異なるのかを考察する．（レポートにはすべてのフィルタサイズの結果を載せる必要はない．適切なものだけを選択して載せること）



## レポート課題3

平均値フィルタの代わりに，以下のフィルタを適用する．

In [ ]:
N = 3
W2 = np.eye(N)
W2 = W2 + np.fliplr(W2)
W2 /= W2.sum()
print(W2)

- 画像`test_pattern.jpg`, `hubble.jpg`を読み込み，それぞれに対してこのフィルタによるフィルタリングを適用する．
- フィルタのサイズ`N`（奇数）を3, 5, ..., 55まで変えて，結果画像のどの部分がどのようになっているのか，なぜそうなるのか，また平均値フィルタとどのように異なるのかを考察する．（レポートにはすべてのフィルタサイズの結果を載せる必要はない．適切なものだけを選択して載せること）








# ===================================


# パート2：大量の画像に対するフィルタリング

以下ではフィルタリングを数万枚の画像に適用する．

* データセット [Cats and Dogs in Kaggle](https://www.kaggle.com/c/dogs-vs-cats/data)のものを利用
* データセット入手先 https://www.microsoft.com/en-us/download/details.aspx?id=54765

以下の手順に従って，画像データセットをダウンロードし，画像を展開する．

In [ ]:
import os
os.makedirs('data', exist_ok=True)  # 現在のディレクトリに'data'フォルダを作成する

zip_file_name = os.path.join('data', 'dog-cat.zip')  # ダウンロードして保存するファイル名を'data/dog-cat.zip'にする

In [ ]:
# ダウンロード用関数
import requests
def download(url, filename):
    with open(filename, 'wb') as saveFile:
        saveFile.write(requests.get(url).content)


# 画像データセットをダウンロードする
download('https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip',
         zip_file_name)  # 820MBあるのでダウンロードに時間がかかります

In [ ]:
# ダウンロードしたzipファイルを展開する
import zipfile
with zipfile.ZipFile(zip_file_name) as zip_file:
    zip_file.extractall('data')  # 2.5万枚あるので展開に時間がかかります

これで，現在のディレクトリの`data/PetImages/Cat/`もしくは`data/PetImages/Dog`以下に，jpgファイルが保存された．

まず，それらのファイル名を取得する．そのために`glob`モジュールを使って，猫の画像ファイルのリスト`cat_files`と犬の画像のファイルのリスト`dog_files`を作成する．`len`でリストの長さを表示すれば分かるように，それぞれ12,500枚の画像がある．

In [ ]:
import glob
cat_files = glob.glob(os.path.join('data', 'PetImages', 'Cat', '*.jpg'))
dog_files = glob.glob(os.path.join('data', 'PetImages', 'Dog', '*.jpg'))
print('There are ', len(cat_files), 'Cat images.')
print('There are ', len(cat_files), 'Dog images.')

それぞれのリストの先頭のファイルを表示して画像を確認する．

In [ ]:
im = imread(cat_files[0])
imshow(im)
plt.title(cat_files[0])
plt.show()

im = imread(dog_files[0])
imshow(im)
plt.title(dog_files[0])
plt.show()

## 確認

画像を保存したフォルダを開いて，多数の犬と猫の画像があることを確認する．

フォルダ中にファイルが1万枚以上あるため，ファイルリストが表示されるのは非常に遅いことを覚悟すること．（マシンスペックの低いPCでは，マシンが反応しなくなり，強制シャットダウンするしかない状況になる可能性もあるので注意すること）

それではこれらのファイル全てに`N=15`の平均値フィルタによるフィルタリングを適用する．

In [ ]:
N = 15
W = np.ones((N, N)) / (N**2)  # N×N平滑化フィルタW

1万枚以上あるファイルをループで処理するため，今何回目のループかを表示するプログレスバーがあると便利である．
ここではそのためのtqdmモジュールを利用する．

In [ ]:
# プログレスバーを表示するためのモジュール
from tqdm.notebook import tqdm

まずは<b>猫</b>の画像に対してフィルタリングを行う．

In [ ]:
# 1. フィルタリングした画像の保存先フォルダを作成
os.makedirs('data_cat', exist_ok=True)  # 現在のディレクトリに'data_cat'フォルダを作成

# 2. すべての猫の画像ファイル名を取得
cat_files = sorted(glob.glob(os.path.join('data', 'PetImages', 'Cat', '*.jpg')), 
                   key=lambda x: int(os.path.basename(os.path.splitext(x)[0])))  # 連番ファイル名順にソートする方法

cat_files = cat_files[:100]  # 最初の100枚だけを使用．すべての画像に適用するにはこの行をコメントアウトする

# 3. 各ファイルについて
for f in tqdm(cat_files):  # プログレスバー表示のためのループ方法

    # 読み込み
    print(f)
    im = imread(f)
   
    # フィルタリング
    im_filtered = np.zeros_like(im)
    for i in range(3):
        im_filtered[..., i] = convolve(im[..., i], W, mode='same')

    # 保存
    save_file_name = os.path.join('data_cat', os.path.basename(f))
    imsave(save_file_name, im_filtered)

## 確認

フィルタリングした画像を保存したフォルダを開いて，処理した100枚に対して平均値フィルタが適用されている（つまりボケている）ことを確認する．





## すべての画像に適用する

### コメントアウト

上記のコードは最初の100枚だけに適用するものである．

`cat_files = cat_files[:100]`

という行をコメントアウトする，つまり先頭に`#`を入れて

`# cat_files = cat_files[:100]`

とすれば，すべての画像に適用できる．


### 不適切な画像の除外

すべての画像に適用した場合，エラーにより途中でループが停止してしまう．
その理由は様々だが，大規模なデータを扱う場合には例外的なデータは必ず発生する．

ここでは，エラーが出て読み込めなかった・変換できなかったファイルを削除することで対応する．
つまり，実行してループがエラーで停止したら，その時のファイル（エラーが出て止まったときに最後に表示されているファイル）
をフォルダから削除する．

これを，エラーが無くなるまで繰り返す．

### 確認

フィルタリングした画像を保存したフォルダを開いて，すべての画像に（削除したファイルを除いて）
平均値フィルタが適用されている（つまりボケている）ことを確認する．



## レポート課題4

- すべての<b>犬</b>の画像に対して，レポート課題2または3のフィルタを適用するコードを作成し，実行する．（フィルタサイズは各自で決定すること）
- エラーとなるファイルを除外すること．またそのファイル・ファイル名を確認する．（どのファイルを除外したのかをレポートに記載すること）
- 何枚かの画像に対して，フィルタリングされていることを確認する．（確認した画像をレポートに掲載すること）



# レポート課題


## 概要

- レポート課題1, 2, 3, 4を行い，そのコードと結果画像をレポートにまとめる．
- 自分のカメラで撮影した画像を`images`フォルダに保存し，その画像に対してレポート課題1, 2, 3を行い，結果画像をレポートにまとめる．



## 評価

- レポートの評価基準
 - レポートの体裁（PDFか，指定したテンプレートを使ったフォーマットか，氏名・学生番号があるか，タイトルは適切か）
 - 課題の説明（どのような課題の結果を記述したレポートなのかが明確に記述できているか）
 - 文章・画像の引用がある場合，その出典を明記しているか
 - コード（課題を説明するために必要十分なコードがコメント付きで掲載されているか，コードの説明が論理的な文章として記述されているか）
 - 処理結果（処理前後の画像を比較して掲載しているか，画像が複数ある場合には，説明を付けてそれらを掲載しているか）
 - 考察（論理的な文章として記述されているか，客観的な結果を示しそれを元に議論しているか）
- 提出方法
 - Bb9にPDFファイルをアップロードする
 - フォーマットは指定のLaTeXテンプレートを使う．[テンプレートはこちら](https://www.overleaf.com/read/sfrdswwcwmyx)
 - ワードファイルは不可（docx形式も，それを変換したPDFも不可）
 